<a href="https://colab.research.google.com/github/zamalex/BakingFinal/blob/master/hapt_lstm_hier_smote_resample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [79]:
import numpy as np
import pandas as pd
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import sklearn.metrics
from tensorflow.keras.utils import plot_model
import datetime


In [80]:
%load_ext tensorboard


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


# Some methods and data which will be used below

In [81]:
# (i).   Mapping labels to their resp. classes
# (ii).  Mapping classes to their resp. labels

label_to_class = {
    1  : 'WALKING',          
    2  : 'WALKING_UPSTAIRS', 
    3  : 'WALKING_DOWNSTAIRS',
    4  : 'SITTING',      
    5  : 'STANDING',       
    6  : 'LAYING',         
    7  : 'STAND_TO_SIT',      
    8  : 'SIT_TO_STAND',     
    9  : 'SIT_TO_LIE',     
    10 : 'LIE_TO_SIT',      
    11 : 'STAND_TO_LIE',      
    12 : 'LIE_TO_STAND',   
    np.nan : np.nan
}
class_to_label = {
    'WALKING' : 1,
    'WALKING_UPSTAIRS' : 2,
    'WALKING_DOWNSTAIRS' : 3,
    'SITTING' : 4,
    'STANDING' : 5,         
    'LAYING' : 6,      
    'STAND_TO_SIT' : 7,     
    'SIT_TO_STAND' : 8,     
    'SIT_TO_LIE' : 9,     
    'LIE_TO_SIT' : 10,        
    'STAND_TO_LIE' : 11,     
    'LIE_TO_STAND' : 12,
    np.nan : np.nan
}

In [82]:
# (i).   Removing data-points where y and x values is null

# Other methods can be
# ffill (forward fill) => fills using forward points
# bfill (backward fill) => using backward points
# interpolate

def remove_null(xdata,ydata):
    xdata = xdata[np.where(np.isfinite(ydata))]
    ydata = ydata[np.where(np.isfinite(ydata))]
    ydata = ydata[np.where(np.isfinite(xdata).all(axis = 1).all(axis = 1).all(axis = 1))]
    xdata = xdata[np.where(np.isfinite(xdata).all(axis = 1).all(axis = 1).all(axis = 1))]
 
    return xdata,ydata

In [83]:
# normalize xdata using sklearn.preprocessing.StandardScaler and returns 
# scaler object to use it furthur for testing data

# Each axis of each sensor has different min, max, I scaled according to them seperately
# Initial shape == (None,128,2,3)
# changed to (None , 6) :-
# reshape to (None,128,6) -> swapaxis(0,2) -> reshape(6,-1) -> transpose
# Fit scaler OR transform according to scaler

# Reverse above process to get back oiginal data
# transpose -> reshape(6,128,None) -> swapaxes(0,2) -> reshape(None,128,2,3)

def get_scaler(xdata):
    
    row = xdata.shape[0]
    timestamp = xdata.shape[1]
    sensor = xdata.shape[2]
    axis = xdata.shape[3]
    
    scaler = MinMaxScaler(feature_range = (-1,1))
    xdata = xdata.reshape(row,timestamp,sensor*axis)
    xdata = np.swapaxes(xdata,0,2).reshape(sensor*axis,-1).T
    scaler.fit(xdata)
    return scaler

def scale_data(xdata,scaler):
    
    row = xdata.shape[0]
    timestamp = xdata.shape[1]
    sensor = xdata.shape[2]
    axis = xdata.shape[3]
    
    xdata = xdata.reshape(row,timestamp,sensor*axis)
    xdata = np.swapaxes(xdata,0,2).reshape(sensor*axis,-1).T
    xdata = scaler.transform(xdata)
    xdata = xdata.T.reshape(sensor*axis,timestamp,row)
    xdata = np.swapaxes(xdata,0,2).reshape(row,timestamp,sensor,axis)
    
    return xdata

In [84]:
# takes in location, exp no., user no., start and end(end point is excluded from reading i.e lastpoint+1) point 
# ,overlap array, and returns xdata and ydata

def create_windows(location, exp, user, start, end, activity, length, overlap):
    
    acc_file  = location + '/acc_exp'+ str(exp).zfill(2) + '_user' + str(user).zfill(2) + '.txt'
    gyro_file = location + '/gyro_exp'+ str(exp).zfill(2) + '_user' + str(user).zfill(2) + '.txt'

    acc_data  = np.loadtxt(acc_file)
    gyro_data = np.loadtxt(gyro_file)
    
    xtrain = []
    ytrain = []
    
    while (start + length <= end) : 
          
        stop = start + length
        window = []
        
        while start != stop :
            window.append( [acc_data[start] , gyro_data[start]] )
            start += 1
        
        xtrain.append(window)
        ytrain.append(activity)
        
        start = stop - overlap[activity-1]

    return xtrain,ytrain

In [85]:
# location == location of file
# lenght == lenght of window
# overlap == array of overlaps of size == number of unique activities
# overlap depends on activity so as to extract more data from a particular class if needed


# (i).   Loading labels.txt as labels
# (ii).  Iterating in labels and calling create_windows on acceleration file, extending returned data in xtrain, ytrain 
# (iii). Iterating in labels and calling create_windows on gyroscope file, extending returned data in xtrain, ytrain 

def prepare_data(location,length = 128,overlap = [64]*12):

    xdata = []
    ydata = []
        
    labels = np.loadtxt(location+'/labels.txt',dtype = 'uint32')
    
    for exp,user,activity,start,end in labels :

        xtemp , ytemp = create_windows(location, exp, user, start, end+1, activity, length, overlap)
        xdata.extend(xtemp)
        ydata.extend(ytemp)
        
    return np.array(xdata),np.array(ydata)

In [86]:
# (i). Finds max element index sets its 1 and sets remaining 0
#      for each row

def to_categorical(ydata):
    
    for i in range(len(ydata)):
        j = ydata[i].argmax()
        for k in range(len(ydata[i])):
            ydata[i][k] = (k == j)
    return ydata

In [87]:
# (i).  OneHotEncoding ydata
# (ii). Converting sparsh matrix ydata into dense form and then matrix into numpy array

def one_hot_encoded(ydata):
    ydata = OneHotEncoder().fit_transform(ydata.reshape(len(ydata),1))
    ydata = np.asarray(ydata.todense())
    return ydata

def splitData(xtrain,ytrain,type):
  if(type=='master'):
    xtrain = xtrain[(ytrain==1) | (ytrain==5)|(ytrain==7)]
    ytrain = ytrain[(ytrain==1) | (ytrain==5)|(ytrain==7)]
  elif(type=='dynamic'):
    xtrain = xtrain[(ytrain==1) | (ytrain==2)|(ytrain==3)]
    ytrain = ytrain[(ytrain==1) | (ytrain==2)|(ytrain==3)]
  elif(type=='static'):
    xtrain = xtrain[(ytrain==4) | (ytrain==5)|(ytrain==6)]
    ytrain = ytrain[(ytrain==4) | (ytrain==5)|(ytrain==6)]
  elif(type=='transition'):
    xtrain = xtrain[(ytrain==7) | (ytrain==8)|(ytrain==9)|(ytrain==10)|(ytrain==11)|(ytrain==12)]
    ytrain = ytrain[(ytrain==7) | (ytrain==8)|(ytrain==9)|(ytrain==10)|(ytrain==11)|(ytrain==12)]
  else:
    xtrain = xtrain
    ytrain = ytrain
  
  xtrain,xtest,ytrain,ytest = train_test_split(xtrain,ytrain,test_size = 0.3,random_state=42)
  testlabels = ytest
  scaler = get_scaler(xtrain)
  xtrain = scale_data(xtrain,scaler)
  xtest  = scale_data(xtest,scaler)

  ytrain = one_hot_encoded(ytrain)
  ytest = one_hot_encoded(ytest)
  return xtrain,xtest,ytrain,ytest,testlabels


def resampleData(x,y):
  x = x.reshape(5773,768)
  df = pd.DataFrame(x)

  from imblearn.over_sampling import RandomOverSampler
  from collections import Counter
  from sklearn.datasets import make_classification
  from imblearn.over_sampling import SMOTE

  oversample = SMOTE()
  df, y = oversample.fit_resample(df, y)
  df=df.to_numpy()
  x=df.reshape(12228,128,6)
  x=df.reshape(12228,128,2,3)
  return x,y

# Data Preparation

## Loading data from files

In [88]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [89]:
# Preparing data, xtrain, ytrain
# Last six classes [7 to 12] has very less weightage in data since they are extra classes added 
# , made from original six classes
# so, I took more overlapping in them to get slightly more data

xtrain,ytrain = prepare_data('/content/drive/MyDrive/HAPT Data Set/RawData',128,[0,0,0,0,0,0,0,0,0,0,0,0])

In [90]:
originalx = xtrain
originaly=ytrain

In [91]:
#xtrain=originalx
#ytrain = originaly

In [92]:
ytrain.shape

(5773,)

In [93]:
xtrain.shape

(5773, 128, 2, 3)

In [94]:
128*6

768

In [95]:
xtrain = xtrain.reshape(5773,768)


In [96]:
xtrain.shape

(5773, 768)

In [97]:
df = pd.DataFrame(xtrain)
df.sample()

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
1713,0.822222,0.023611,0.247222,0.249844,0.277027,0.01405,0.863889,0.002778,0.236111,0.130114,...,0.055556,-0.671952,-0.253815,0.175318,1.670833,-0.263889,0.413889,-0.986853,1.099252,0.54031


In [98]:
ytrain.shape

(5773,)

In [99]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [100]:
import tensorflow as tf
import datetime

# Clear any logs from previous runs
!rm -rf ./logs/ 

In [101]:
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE

oversample = SMOTE()
df, ytrain = oversample.fit_resample(df, ytrain)

In [102]:
xtrain.shape

(5773, 768)

In [103]:
df.shape

(12228, 768)

In [104]:
ytrain.shape

(12228,)

In [105]:
df=df.to_numpy()
xtrain=df.reshape(12228,128,6)

In [106]:
xtrain.shape

(12228, 128, 6)

In [107]:
xtrain=df.reshape(12228,128,2,3)

In [108]:
xtrain.shape

(12228, 128, 2, 3)

**start SVM**

In [109]:
X_train = pd.read_csv('/content/drive/MyDrive/newdata48features.csv', header=None)
xxx,Y_train = resampleData(originalx,originaly)

In [110]:
X_train.head() # mean ,median , std , variance , root mean square , max, min  

,0,1,2,3,4,5,6,7,8,9,...,38,39,40,41,42,43,44,45,46,47
0,0__median,0__mean,0__standard_deviation,0__variance,0__root_mean_square,0__maximum,0__absolute_maximum,0__minimum,1__median,1__mean,...,4__absolute_maximum,4__minimum,5__median,5__mean,5__standard_deviation,5__variance,5__root_mean_square,5__maximum,5__absolute_maximum,5__minimum
1,1.01944450285877,1.0193142942447153,0.0024762469750240646,6.131799081315831e-06,1.01931730204617,1.027777854158295,1.027777854158295,1.0125000434425,-0.1250000020616516,-0.12428385819818219,...,0.02138028293848038,-0.02138028293848038,0.003054326167330146,0.0024673228924712025,0.006133609869028267,3.762117002544095e-05,0.006611267070777992,0.01679879426956177,0.01679879426956177,-0.01404990069568157
2,1.01944450285877,1.0199002330079632,0.0033669412028527372,1.1336293063467436e-05,1.0199057905428133,1.02916674604155,1.02916674604155,1.009722259675992,-0.1263888939449057,-0.12505425802505304,...,0.03451388701796532,-0.03451388701796532,0.00366519158706069,0.00544290474636,0.009781032110216805,9.566858914109219e-05,0.011193471365891834,0.03634648025035858,0.03634648025035858,-0.01435533352196217
3,1.01944450285877,1.0201497995182354,0.002750501804412125,7.565260176074354e-06,1.0201535074278145,1.025000070391787,1.025000070391787,1.011111151559246,-0.1305555574387036,-0.13018663702089006,...,0.02626720629632473,-0.02626720629632473,0.0,0.0003436116994635086,0.009007026194748656,8.112652087288845e-05,0.009013578083807598,0.02718350291252136,0.02718350291252136,-0.02382374368607998
4,1.020833394742025,1.0207791411528349,0.0028706455352721085,8.24060578917769e-06,1.0207831775742136,1.027777854158295,1.027777854158295,1.013888935325754,-0.1263888939449057,-0.1276367229824334,...,0.03390302136540413,-0.03390302136540413,0.005345070967450738,0.006118197258047076,0.012574714847580413,0.00015812345349795927,0.013984126400541938,0.03359758853912354,0.03359758853912354,-0.02443460933864117


In [111]:
Y_train.shape

(12228,)

In [112]:
X_train=X_train.iloc[1: , :]


In [113]:
X_train,X_test,Y_train,Y_test = train_test_split(X_train,Y_train,test_size = 0.3,random_state=42)


In [114]:
X_train.shape,X_test.shape,Y_train.shape,Y_test.shape

((8559, 48), (3669, 48), (8559,), (3669,))

In [115]:
from sklearn.svm import LinearSVC, SVC
from sklearn.metrics import confusion_matrix, accuracy_score


svc = LinearSVC(random_state=42, tol=1e-5, verbose=1, max_iter=10000)
svc.fit(X_train, Y_train)

svm_pred = svc.predict(X_test)
SVC_pred_test = accuracy_score(Y_test, svm_pred)*100
print('Accuracy score TEST: {:.3f}'.format(accuracy_score(Y_test, svm_pred)*100))

[LibLinear]Accuracy score TEST: 90.651


In [116]:
svm_pred.shape

(3669,)

**end** **svm**

## Handling missing data and Normalizing data 

In [117]:
xtrain,ytrain = remove_null(xtrain,ytrain)

In [118]:
xtrain.shape

(12228, 128, 2, 3)

In [119]:
2449+2960+364

5773

In [120]:
ytrain.shape

(12228,)

**Hier split and training**

**Dynamic**

In [121]:
dynamicxtrain,dynamicxtest,dynamicytrain,dynamicytest,dynamiclabels = splitData(xtrain,ytrain,'dynamic')

In [122]:
dynamicxtrain.shape,dynamicytrain.shape

((2139, 128, 2, 3), (2139, 3))

In [123]:
####################################################################################################################################

In [124]:
dynamicxtrain.shape,dynamicxtest.shape,dynamicytrain.shape,dynamicytest.shape,dynamiclabels.shape

((2139, 128, 2, 3), (918, 128, 2, 3), (2139, 3), (918, 3), (918,))

In [125]:
dynamicxtrain = dynamicxtrain.reshape(2139,128,6)
dynamicxtest = dynamicxtest.reshape(918,128,6)

In [126]:
# fit and evaluate a model
# lstm model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import to_categorical
from matplotlib import pyplot

timesteps = len(dynamicxtrain[0])
input_dim = len(dynamicxtrain[0][0])
n_classes =3

print(timesteps)
print(input_dim)
print(len(dynamicxtrain))



128
6
2139


In [127]:
epochs = 100
batch_size = 16
n_hidden = 64
model1 = Sequential()
# Configuring the parameters
model1.add(LSTM(n_hidden, input_shape=(timesteps, input_dim), return_sequences=True))
# Adding a dropout layer
model1.add(Dropout(0.2))
model1.add(LSTM(32))
model1.add(Dropout(0.2))
# Adding a dense output layer with sigmoid activation
model1.add(Dense(n_classes, activation='sigmoid'))
model1.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 128, 64)           18176     
                                                                 
 dropout_6 (Dropout)         (None, 128, 64)           0         
                                                                 
 lstm_7 (LSTM)               (None, 32)                12416     
                                                                 
 dropout_7 (Dropout)         (None, 32)                0         
                                                                 
 dense_3 (Dense)             (None, 3)                 99        
                                                                 
Total params: 30,691
Trainable params: 30,691
Non-trainable params: 0
_________________________________________________________________


In [128]:
model1.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


In [129]:
model1.fit(dynamicxtrain,
          dynamicytrain,
          batch_size=batch_size,
          #validation_data=(dynamicxtest, dynamicytest),
          epochs=epochs,
          )

Epoch 1/100
134/134 [==============================] - 6s 12ms/step - loss: 0.8232 - accuracy: 0.6110
Epoch 2/100
134/134 [==============================] - 1s 10ms/step - loss: 0.6047 - accuracy: 0.7340
Epoch 3/100
134/134 [==============================] - 1s 10ms/step - loss: 0.4823 - accuracy: 0.8111
Epoch 4/100
134/134 [==============================] - 1s 10ms/step - loss: 0.3373 - accuracy: 0.8841
Epoch 5/100
134/134 [==============================] - 1s 10ms/step - loss: 0.3608 - accuracy: 0.8939
Epoch 6/100
134/134 [==============================] - 1s 10ms/step - loss: 0.2849 - accuracy: 0.9046
Epoch 7/100
134/134 [==============================] - 1s 10ms/step - loss: 0.4155 - accuracy: 0.8597
Epoch 8/100
134/134 [==============================] - 1s 10ms/step - loss: 0.2546 - accuracy: 0.9275
Epoch 9/100
134/134 [==============================] - 2s 14ms/step - loss: 0.4670 - accuracy: 0.7990
Epoch 10/100
134/134 [==============================] - 2s 14ms/step - loss: 0.729

In [130]:
predict_x=model1.predict(dynamicxtest) 
ypred=np.argmax(predict_x,axis=1)

29/29 [==============================] - 1s 5ms/step


In [131]:
ypred.shape, dynamiclabels.shape

((918,), (918,))

In [132]:

ypred[ypred == 2] = 3
ypred[ypred == 1] = 2
ypred[ypred == 0] = 1
ypred

array([3, 2, 3, 3, 1, 2, 3, 3, 1, 2, 2, 3, 3, 3, 1, 1, 1, 3, 1, 3, 3, 2,
       1, 1, 1, 2, 2, 3, 2, 3, 3, 2, 3, 1, 2, 2, 1, 1, 3, 2, 1, 3, 3, 1,
       2, 1, 2, 3, 1, 3, 3, 2, 3, 2, 2, 3, 1, 3, 1, 3, 3, 1, 2, 2, 3, 3,
       2, 3, 2, 1, 2, 2, 3, 3, 2, 2, 1, 2, 3, 2, 2, 2, 1, 3, 1, 3, 2, 2,
       2, 3, 3, 3, 3, 3, 2, 1, 3, 1, 3, 1, 2, 2, 1, 2, 2, 1, 1, 3, 2, 2,
       3, 1, 1, 2, 1, 2, 3, 3, 3, 3, 3, 1, 1, 3, 2, 2, 1, 1, 1, 2, 1, 2,
       3, 1, 1, 2, 3, 1, 2, 2, 1, 2, 3, 2, 3, 3, 2, 1, 3, 2, 2, 1, 2, 3,
       1, 1, 1, 1, 1, 2, 2, 3, 2, 2, 2, 1, 1, 1, 1, 2, 3, 1, 2, 3, 1, 3,
       2, 2, 3, 2, 1, 3, 3, 1, 3, 2, 2, 3, 2, 2, 2, 3, 2, 3, 2, 3, 1, 2,
       1, 3, 1, 2, 2, 1, 1, 2, 2, 1, 2, 2, 2, 1, 1, 2, 2, 1, 2, 3, 3, 1,
       2, 3, 3, 3, 1, 2, 1, 3, 3, 3, 3, 3, 1, 2, 2, 1, 1, 1, 1, 3, 3, 3,
       1, 2, 2, 2, 3, 3, 1, 2, 1, 3, 2, 3, 2, 2, 2, 3, 3, 2, 2, 1, 1, 1,
       2, 2, 3, 2, 3, 3, 1, 1, 2, 2, 1, 3, 1, 1, 2, 3, 2, 2, 1, 2, 3, 3,
       3, 3, 1, 3, 3, 3, 3, 1, 3, 1, 1, 1, 2, 2, 1,

In [133]:
from sklearn.metrics import confusion_matrix, accuracy_score

print('Accuracy score TEST: {:.3f}'.format(accuracy_score(ypred, dynamiclabels)*100))

Accuracy score TEST: 99.455


**static**

In [134]:
staticxtrain,staticxtest,staticytrain,staticytest,staticlabels = splitData(xtrain,ytrain,'static')
staticxtrain.shape,staticxtest.shape,staticytrain.shape,staticytest.shape,staticlabels.shape

((2139, 128, 2, 3), (918, 128, 2, 3), (2139, 3), (918, 3), (918,))

In [135]:
staticxtrain =staticxtrain.reshape(2139,128,6)
staticxtest = staticxtest.reshape(918,128,6)

In [136]:
# fit and evaluate a model
# lstm model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import to_categorical
from matplotlib import pyplot

timesteps = len(staticxtrain[0])
input_dim = len(staticxtrain[0][0])
n_classes =3

print(timesteps)
print(input_dim)
print(len(staticxtrain))

epochs = 100
batch_size = 16
n_hidden = 64
model2 = Sequential()
# Configuring the parameters
model2.add(LSTM(n_hidden, input_shape=(timesteps, input_dim), return_sequences=True))
# Adding a dropout layer
model2.add(Dropout(0.2))
model2.add(LSTM(32))
model2.add(Dropout(0.2))
# Adding a dense output layer with sigmoid activation
model2.add(Dense(n_classes, activation='sigmoid'))
model2.summary()

128
6
2139
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_8 (LSTM)               (None, 128, 64)           18176     
                                                                 
 dropout_8 (Dropout)         (None, 128, 64)           0         
                                                                 
 lstm_9 (LSTM)               (None, 32)                12416     
                                                                 
 dropout_9 (Dropout)         (None, 32)                0         
                                                                 
 dense_4 (Dense)             (None, 3)                 99        
                                                                 
Total params: 30,691
Trainable params: 30,691
Non-trainable params: 0
_________________________________________________________________


In [137]:
model2.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


In [138]:
model2.fit(staticxtrain,
          staticytrain,
          batch_size=batch_size,
          #validation_data=(staticxtest, staticytest),
          epochs=epochs,
          )

Epoch 1/100
134/134 [==============================] - 6s 10ms/step - loss: 0.4434 - accuracy: 0.7999
Epoch 2/100
134/134 [==============================] - 1s 10ms/step - loss: 0.3041 - accuracy: 0.8756
Epoch 3/100
134/134 [==============================] - 1s 10ms/step - loss: 0.2657 - accuracy: 0.8897
Epoch 4/100
134/134 [==============================] - 1s 10ms/step - loss: 0.2815 - accuracy: 0.8850
Epoch 5/100
134/134 [==============================] - 1s 10ms/step - loss: 0.2575 - accuracy: 0.9004
Epoch 6/100
134/134 [==============================] - 1s 10ms/step - loss: 0.2642 - accuracy: 0.8920
Epoch 7/100
134/134 [==============================] - 1s 10ms/step - loss: 0.2434 - accuracy: 0.8957
Epoch 8/100
134/134 [==============================] - 2s 13ms/step - loss: 0.2401 - accuracy: 0.8995
Epoch 9/100
134/134 [==============================] - 2s 14ms/step - loss: 0.2382 - accuracy: 0.9056
Epoch 10/100
134/134 [==============================] - 1s 10ms/step - loss: 0.235

**transition**

In [139]:
transitionxtrain,transitionxtest,transitionytrain,transitionytest,transitionlabels = splitData(xtrain,ytrain,'transition')
transitionxtrain.shape,transitionxtest.shape,transitionytrain.shape,transitionytest.shape,transitionlabels.shape

((4279, 128, 2, 3), (1835, 128, 2, 3), (4279, 6), (1835, 6), (1835,))

In [140]:
transitionxtrain =transitionxtrain.reshape(4279,128,6)
transitionxtest = transitionxtest.reshape(1835,128,6)

In [141]:
# fit and evaluate a model
# lstm model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import to_categorical
from matplotlib import pyplot

timesteps = len(transitionxtrain[0])
input_dim = len(transitionxtrain[0][0])
n_classes =6

print(timesteps)
print(input_dim)
print(len(transitionxtrain))

epochs = 100
batch_size = 16
n_hidden = 64
model3 = Sequential()
# Configuring the parameters
model3.add(LSTM(n_hidden, input_shape=(timesteps, input_dim), return_sequences=True))
# Adding a dropout layer
model3.add(Dropout(0.2))
model3.add(LSTM(32))
model3.add(Dropout(0.2))# Adding a dense output layer with sigmoid activation
model3.add(Dense(n_classes, activation='sigmoid'))
model3.summary()

128
6
4279
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_10 (LSTM)              (None, 128, 64)           18176     
                                                                 
 dropout_10 (Dropout)        (None, 128, 64)           0         
                                                                 
 lstm_11 (LSTM)              (None, 32)                12416     
                                                                 
 dropout_11 (Dropout)        (None, 32)                0         
                                                                 
 dense_5 (Dense)             (None, 6)                 198       
                                                                 
Total params: 30,790
Trainable params: 30,790
Non-trainable params: 0
_________________________________________________________________


In [142]:
model3.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


In [143]:
model3.fit(transitionxtrain,
          transitionytrain,
          batch_size=batch_size,
          #validation_data=(transitionxtest, transitionytest),
          epochs=epochs,
          )

Epoch 1/100
268/268 [==============================] - 7s 10ms/step - loss: 0.9807 - accuracy: 0.5747
Epoch 2/100
268/268 [==============================] - 3s 10ms/step - loss: 0.7136 - accuracy: 0.6824
Epoch 3/100
268/268 [==============================] - 3s 10ms/step - loss: 0.5971 - accuracy: 0.7464
Epoch 4/100
268/268 [==============================] - 3s 11ms/step - loss: 0.4779 - accuracy: 0.8114
Epoch 5/100
268/268 [==============================] - 3s 12ms/step - loss: 0.4328 - accuracy: 0.8404
Epoch 6/100
268/268 [==============================] - 3s 10ms/step - loss: 0.4135 - accuracy: 0.8490
Epoch 7/100
268/268 [==============================] - 3s 10ms/step - loss: 0.3766 - accuracy: 0.8591
Epoch 8/100
268/268 [==============================] - 3s 10ms/step - loss: 0.3491 - accuracy: 0.8722
Epoch 9/100
268/268 [==============================] - 3s 12ms/step - loss: 0.3003 - accuracy: 0.8953
Epoch 10/100
268/268 [==============================] - 3s 12ms/step - loss: 0.275

In [144]:
predict_x=model3.predict(transitionxtest) 
ypred=np.argmax(predict_x,axis=1)


ypred[ypred == 5] = 12
ypred[ypred == 4] = 11
ypred[ypred == 3] = 10
ypred[ypred == 2] = 9
ypred[ypred == 1] = 8
ypred[ypred == 0] = 7
ypred

58/58 [==============================] - 1s 6ms/step


array([12, 11, 10, ..., 12, 10, 10])

In [145]:
from sklearn.metrics import confusion_matrix, accuracy_score

print('Accuracy score TEST: {:.3f}'.format(accuracy_score(ypred, transitionlabels)*100))

Accuracy score TEST: 99.292


**Final**

In [146]:
xtrain,ytrain = resampleData(originalx,originaly)

xtrain,xtest,ytrain,ytest,ylabels = splitData(xtrain,ytrain,'all')
xtrain.shape,ytrain.shape,xtest.shape,ytest.shape

((8559, 128, 2, 3), (8559, 12), (3669, 128, 2, 3), (3669, 12))

In [147]:
scores=[]
dynamics = np.array([1,2,3])
statics = np.array([4,5,6])
transitions = np.array([7,8,9,10,11,12])
for i in range(len(xtest)):
    row = xtest[i].reshape(1,128,6)
    if(svm_pred[i] in [1,2,3]):
        predict_x=model1.predict(row) 
        
        pred=dynamics[np.argmax(predict_x,axis=1)]
        scores.append(pred)
    elif(svm_pred[i] in [4,5,6]):
        predict_x=model2.predict(row) 
        
        pred=statics[np.argmax(predict_x,axis=1)]
        scores.append(pred)
    else:
        predict_x=model3.predict(row) 
    
        pred=transitions[np.argmax(predict_x,axis=1)]
        scores.append(pred)
    print(i)

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 23ms/step
1169
1/1 [==============================] - 0s 21ms/step
1170
1/1 [==============================] - 0s 22ms/step
1171
1/1 [==============================] - 0s 22ms/step
1172
1/1 [==============================] - 0s 21ms/step
1173
1/1 [==============================] - 0s 24ms/step
1174
1/1 [==============================] - 0s 22ms/step
1175
1/1 [==============================] - 0s 23ms/step
1176
1/1 [==============================] - 0s 22ms/step
1177
1/1 [==============================] - 0s 29ms/step
1178
1/1 [==============================] - 0s 21ms/step
1179
1/1 [==============================] - 0s 22ms/step
1180
1/1 [==============================] - 0s 23ms/step
1181
1/1 [==============================] - 0s 22ms/step
1182
1/1 [==============================] - 0s 24ms/step
1183
1/1 [==============================] - 0s 24ms/step
1184
1/1 [==============================] 

In [148]:
scores=np.array(scores)
scores.shape

(3669, 1)

In [149]:
ylabels.shape

(3669,)

In [150]:

print('Accuracy score TEST: {:.3f}'.format(accuracy_score(ylabels, scores)*100))

Accuracy score TEST: 65.876
